In [11]:
import sys
sys.path.append('/root/lib/')
import os
import ROOT as r
import pandas as pd
import json
import shutil
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

%jsroot on

In [12]:

shutil.copy('/home/xinduan/桌面/MilllliQan_json/mqLumis.json', os.getcwd())
mqLumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')
beam_0_data = mqLumis[mqLumis['beam'] == 0]
unique_runs = beam_0_data['run'].unique()
print("Unique run values with beam = 0:")
print(unique_runs)
#beam_in_fill_true = mqLumis[mqLumis['beamInFill'] == True]
#runs_with_beam_in_fill_true = beam_in_fill_true['run'].unique()
#print("Run values with beamInFill = True:")
#print(runs_with_beam_in_fill_true)
mqLumis[3370:]

Unique run values with beam = 0:
[1006 1007 1026 1031 1032 1033 1034 1035 1036 1037 1038 1039 1040 1041
 1042 1043 1044 1045 1046 1047 1048 1049 1050 1051 1052 1053 1054 1055
 1056 1057 1058 1114 1116 1117 1118 1119 1120 1121 1122 1123 1124 1125
 1126 1127 1128 1129 1130 1131 1132 1133 1134 1135 1136 1137 1138 1139
 1140 1141 1142 1143 1144 1145 1146 1147 1148 1149 1150 1151 1152 1153
 1154 1155 1156 1157 1158 1159 1160 1161 1162 1163 1164 1165 1166 1167
 1168 1169 1170 1171 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181
 1182 1183 1184 1185 1186 1187 1188 1189 1190 1191 1192 1193 1194 1195
 1196 1197 1198 1199 1200 1201 1202 1203 1204 1205 1206 1207 1208 1209
 1210 1211 1212 1213 1214 1215 1216 1217 1218 1219 1220 1221 1222 1223
 1224 1225 1226 1227 1228 1229 1230 1231 1232 1233 1234 1235 1236 1237
 1238 1239 1240 1241 1242 1243 1244 1245 1246 1247 1248 1249 1250 1251
 1252 1253 1254 1255 1256 1257 1258 1259 1260 1261 1262 1263 1264 1265
 1266 1267 1268 1269 1270 1271 1272 1273 127

,run,file,lumis,fill,beam,beamInFill,dir,filename,start,stop,beamType,beamEnergy,betaStar,fillStart,fillEnd,startStableBeam,endStableBeam,lumiEst
3370,1010,6,819.258949,8880,1.0,True,/store/user/milliqan/run3/bar//1000/0001,MilliQan_Run1010.6_default.root,2023-06-06T08:45:51.000Z,2023-06-06T08:46:12.000Z,PROTONS,6799.44,120.0,2023-06-06T02:29:26.000Z,2023-06-06T19:52:56.000Z,2023-06-06T03:46:56.000Z,2023-06-06T19:50:17.000Z,0.297649
3371,1010,7,819.258949,8880,1.0,True,/store/user/milliqan/run3/bar//1000/0001,MilliQan_Run1010.7_default.root,2023-06-06T08:46:13.000Z,2023-06-06T08:46:37.000Z,PROTONS,6799.44,120.0,2023-06-06T02:29:26.000Z,2023-06-06T19:52:56.000Z,2023-06-06T03:46:56.000Z,2023-06-06T19:50:17.000Z,0.340171
3372,1010,8,819.258949,8880,1.0,True,/store/user/milliqan/run3/bar//1000/0001,MilliQan_Run1010.8_default.root,2023-06-06T08:46:37.000Z,2023-06-06T08:47:01.000Z,PROTONS,6799.44,120.0,2023-06-06T02:29:26.000Z,2023-06-06T19:52:56.000Z,2023-06-06T03:46:56.000Z,2023-06-06T19:50:17.000Z,0.340171
3373,1010,9,819.258949,8880,1.0,True,/store/user/milliqan/run3/bar//1000/0001,MilliQan_Run1010.9_default.root,2023-06-06T08:47:02.000Z,2023-06-06T08:47:26.000Z,PROTONS,6799.44,120.0,2023-06-06T02:29:26.000Z,2023-06-06T19:52:56.000Z,2023-06-06T03:46:56.000Z,2023-06-06T19:50:17.000Z,0.340171
3374,1010,10,819.258949,8880,1.0,True,/store/user/milliqan/run3/bar//1000/0001,MilliQan_Run1010.10_default.root,2023-06-06T08:47:26.000Z,2023-06-06T08:47:50.000Z,PROTONS,6799.44,120.0,2023-06-06T02:29:26.000Z,2023-06-06T19:52:56.000Z,2023-06-06T03:46:56.000Z,2023-06-06T19:50:17.000Z,0.340171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70743,1609,122,0,None,0.0,False,/store/user/milliqan/run3/bar//1600/0000,MilliQan_Run1609.122_default.root,2024-06-05T21:17:52.000Z,2024-06-05T21:25:03.000Z,None,None,None,None,None,None,None,0.000000
70744,1609,123,0,None,0.0,False,/store/user/milliqan/run3/bar//1600/0000,MilliQan_Run1609.123_default.root,2024-06-05T21:25:04.000Z,2024-06-05T21:32:56.000Z,None,None,None,None,None,None,None,0.000000
70745,1609,124,0,None,0.0,False,/store/user/milliqan/run3/bar//1600/0000,MilliQan_Run1609.124_default.root,2024-06-05T21:32:57.000Z,2024-06-05T21:40:05.000Z,None,None,None,None,None,None,None,0.000000
70746,1609,125,0,None,0.0,False,/store/user/milliqan/run3/bar//1600/0000,MilliQan_Run1609.125_default.root,2024-06-05T21:40:05.000Z,2024-06-05T21:47:29.000Z,None,None,None,None,None,None,None,0.000000


In [13]:
def checkBeam(mqLumis, filename):
    run = int(filename.split('Run')[1].split('.')[0])
    file = int(filename.split('.')[1].split('_')[0]) 
    return mqLumis['beam'].loc[(mqLumis['run'] == run) & (mqLumis['file'] == file)].values[0]

In [38]:
#create root TChain of files for a given run

run = 1580
dataDir = '/home/xinduan/桌面/MilliQan_Run1580/'

#create tchain that will read in tree with name t
mychain = r.TChain('t') 

#add files to mychain
for filename in os.listdir(dataDir):
    if not filename.endswith('.root'): continue
    if 'Run{}'.format(run) in filename:
        print("Adding file {} to the chain".format(filename))
        if checkBeam(mqLumis, filename):
            mychain.Add(dataDir+filename)

#print the number of entries in my tchain
entries = mychain.GetEntries()
print("There are {} total entries in the chain".format(entries))


Adding file MilliQan_Run1580.72_v34.root to the chain
Adding file MilliQan_Run1580.23_v34.root to the chain
Adding file MilliQan_Run1580.104_v34.root to the chain
Adding file MilliQan_Run1580.32_v34.root to the chain
Adding file MilliQan_Run1580.132_v34.root to the chain
Adding file MilliQan_Run1580.151_v34.root to the chain
Adding file MilliQan_Run1580.120_v34.root to the chain
Adding file MilliQan_Run1580.62_v34.root to the chain
Adding file MilliQan_Run1580.129_v34.root to the chain
Adding file MilliQan_Run1580.150_v34.root to the chain
Adding file MilliQan_Run1580.45_v34.root to the chain
Adding file MilliQan_Run1580.127_v34.root to the chain
Adding file MilliQan_Run1580.99_v34.root to the chain
Adding file MilliQan_Run1580.81_v34.root to the chain
Adding file MilliQan_Run1580.68_v34.root to the chain
Adding file MilliQan_Run1580.134_v34.root to the chain
Adding file MilliQan_Run1580.184_v34.root to the chain
Adding file MilliQan_Run1580.159_v34.root to the chain
Adding file MilliQ

In [39]:
#select only the events with hits in front and back slab with area > 100k


mychain.Draw(">>eList", "MaxIf$(area, chan==74) > 100000 && MaxIf$(area, chan==75) > 100000", "entryList")
eList = r.gDirectory.Get("eList")
mychain.SetEntryList(eList)

In [40]:
'''
#create a canvas and draw the time difference 
c1 = r.TCanvas("c1", "c1", 600, 600)
mychain.Draw("MaxIf$(timeFit_module_calibrated, chan==75&&area==MaxIf$(area,chan==75)) - MaxIf$(timeFit_module_calibrated, chan==74&&area==MaxIf$(area,chan==74)) >> h_timeDiff")
#c1.Draw()

h_timeDiff = r.gDirectory.Get("h_timeDiff")

c1.Draw()
'''

# Create a canvas and draw the time difference
c1 = r.TCanvas("c1", "c1", 600, 600)
mychain.Draw("MaxIf$(timeFit_module_calibrated, chan==75&&area==MaxIf$(area,chan==75)) - MaxIf$(timeFit_module_calibrated, chan==74&&area==MaxIf$(area,chan==74)) >> h_timeDiff")

# Get the histogram
h_timeDiff = r.gDirectory.Get("h_timeDiff")
# Define a double Gaussian fit function
fitFunc = r.TF1("fitFunc", "gaus(0) + gaus(3)",h_timeDiff.GetXaxis().GetXmin(), h_timeDiff.GetXaxis().GetXmax())
fitFunc.SetParameters(16.9627, -3.9584, 2.4559, 612043, 874.916, -87.233)  # Initial parameters for the fit

# Fit the histogram
h_timeDiff.Fit(fitFunc)

# Draw the histogram and the fit function
h_timeDiff.Draw()
fitFunc.Draw("SAME")

# Update the canvas
c1.Update()
c1.Draw()


****************************************
         Invalid FitResult  (status = 2 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      29.5777
NDf                       =           33
Edm                       =  2.81874e-08
NCalls                    =           55
p0                        =      16.9624   +/-   0           
p1                        =     -3.95859   +/-   0           
p2                        =      2.45592   +/-   0           
p3                        =       612043   +/-   0           
p4                        =      874.916   +/-   0           
p5                        =     -85.9245   +/-   0           


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <Fit>: Abnormal termination of minimization.


In [41]:
from ROOT import gROOT
gROOT.GetListOfCanvases().Draw()

In [20]:
#save the output in a root file

#get all files from the gDirectory
h_timeDiff = r.gDirectory.Get('h_timeDiff')

#create the output root file
fout = r.TFile("timingCorrectionsOutput.root", 'RECREATE')

#switch to our root file for writing
fout.cd()

#write the plot to the file
h_timeDiff.Write()

#close the file
fout.Close()

